# Chapter 15: Stress Tests


In this chapter, we explore the concept of stress tests and their application in financial institutions. Stress tests are essential tools used to evaluate the resilience and stability of financial institutions under adverse economic scenarios. We begin with a general overview of stress tests, outlining their key elements. Following this, we discuss stressed Value at Risk (VaR) and stressed Expected Shortfall (ES), which are crucial measures for determining companies' required capital positions during stressful situations. Lastly, we examine stress testing frameworks mandated by the US Federal Reserve, including the Comprehensive Capital Analysis and Review (CCAR) and the Dodd-Frank Act Stress Testing (DFAST). This chapter aims to provide a thorough understanding of stress tests and their significance in assessing the robustness of financial institutions during challenging economic conditions.

Chapter Objectives:

1. Introduce stress tests: overview and key elements
2. Define stress VaR and stress ES
3. Discuss stress tests in the United States and Europe

In [1]:
from pandas_datareader import data as pdr
import yfinance
yfinance.pdr_override()
import warnings
warnings.filterwarnings("ignore")

# Set the start and end dates
start_date = "2008-07-01" # 1
end_date = "2009-06-29"
# Retieve prices # 2
SPY = pdr.get_data_yahoo("SPY", start = start_date, end = end_date)
LQD = pdr.get_data_yahoo("LQD", start = start_date, end = end_date)
HYG = pdr.get_data_yahoo("HYG", start = start_date, end = end_date)

# Calculate returns # 3
SPY['ret_spy'] =(SPY['Adj Close']/SPY['Adj Close'].shift(1))
LQD['ret_lqd'] =(LQD['Adj Close']/LQD['Adj Close'].shift(1))
HYG['ret_hyg'] =(HYG['Adj Close']/HYG['Adj Close'].shift(1))

# Merge the three datasets, keep only returns # 4
df  = SPY[['ret_spy']].merge(LQD[['ret_lqd']],left_index=True, right_index=True)
df  = df.merge(HYG[['ret_hyg']],left_index=True, right_index=True)
df = df.iloc[1: , :]
# Calculate portfolio value and gain/loss in each scenario # 5
df['port_value'] = 0.4*df['ret_spy'] + 0.3*df['ret_lqd'] + 0.3*df['ret_hyg']
df['gain_loss'] = df['port_value'] - 1
# Calcuate one percentile value # 6
Stressed_VaR = df['gain_loss'].quantile([0.01])
print("the firm's stressed VaR is", round(Stressed_VaR[0.01],4))


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
the firm's stressed VaR is -0.0481


In [2]:
# Keep obsrevations that the gain/loss is beyond VaR
df_es = df[df["gain_loss"]<Stressed_VaR[0.01]]
print(df_es)
Stressed_ES = df_es["gain_loss"].mean()
# Print out the value
print("the firm’s stressed ES is", round(Stressed_ES,4))


             ret_spy   ret_lqd   ret_hyg  port_value  gain_loss
Date                                                           
2008-09-17  0.955037  0.954189  0.939941    0.950254  -0.049746
2008-09-29  0.921639  0.908889  0.919026    0.917030  -0.082970
2008-10-15  0.901552  1.000000  0.956363    0.947530  -0.052470
the firm’s stressed ES is -0.0617
